### Week 1 - Basic Data Structures

#### Check Brackets in the code

In [88]:
from collections import namedtuple

Bracket = namedtuple("Bracket", ["char", "position"])

def are_matching(left, right):
    return (left + right) in ["()", "[]", "{}"]


def find_mismatch(text):
    opening_brackets_stack = []
    for i, char in enumerate(text):
        if char in "([{":
            # Process opening bracket
            Bracket = (char, i+1) 
            opening_brackets_stack.append(Bracket)

        if char in ")]}":
            # Process closing bracket
            if opening_brackets_stack == []:
                return i + 1
            top = opening_brackets_stack.pop()
            if not are_matching(top[0], char):                
                return i + 1
    if opening_brackets_stack == []:
        return "Success"
    else:
        return opening_brackets_stack[0][1]


In [89]:
find_mismatch("[](()")

3

In [91]:
count = 1
for (text, answer) in [
    ("[]", "Success"),
    ("{}[]", "Success"),
    ("[()]", "Success"),
    ("(())", "Success"),
    ("{[]}()", "Success"),
    ("{", 1),
    ("}", 1),
    (" [}]", 3),
    ("foo(bar);", "Success"),
    ("foo(bar[i);",10 ),
    ("[](()", 3)
    ]:
    print("Test: ", count, text, find_mismatch(text) == answer)
    count += 1

Test:  1 [] True
Test:  2 {}[] True
Test:  3 [()] True
Test:  4 (()) True
Test:  5 {[]}() True
Test:  6 { True
Test:  7 } True
Test:  8  [}] True
Test:  9 foo(bar); True
Test:  10 foo(bar[i); True
Test:  11 [](() True


### Arrays


### Lists 

In [53]:
class sList(namedtuple('sList','key next'), head, tail):
    """
        Singly linked list with tail
    """
    def __init___(self, head, tail):
        
        
    def pushfront(key):
        continue
        
    def popfront(key):
        if self.head == None:
            return "Error, empty list"
        
        
        
        continue
        
    def pushback(key):
        continue
        
    def popfront():
        continue
    
    def topback():
        continue
    
    popback
    
    find
    
    erase
    
    empty
    
    addbefore
    
    addafter
    
        

IndentationError: expected an indented block (1968137059.py, line 6)

In [22]:
#Singly linked lists

In [54]:
from collections import namedtuple

Node = namedtuple('Node','key next')

In [63]:
node1 = Node(7,10)
node2 = Node(10,None)
linkedlist = []


In [64]:
linkedlist.append(node1)
print(linkedlist)

[Node(key=7, next=10)]


In [65]:
linkedlist.append(node2)
print(linkedlist)

[Node(key=7, next=10), Node(key=10, next=None)]


In [ ]:
def push_front(key):
    node = Node(None, None)

In [66]:
class Node:
    """
        Single Node of a linked list
    """
    # constructor function
    def __init__(self, key, next=None):
        self.key = key
        self.next = next
    
class LinkedList:
    """
        Linked List class with a single node as head and tail
    """
    def __init__(self):
        self.head = None
        self.tail = None
    

In [67]:
first = Node(3)

In [69]:
print(first.key)

3
